В задаче необходимо научиться определять, утонет ли посетитель Титаника или нет по наличию (не всегда хорошо заполненных) признаков.

train.csv — набор данных на основании которого будет строиться модель (обучающая выборка)<br>
test.csv — набор данных для проверки модели

Для каждого пассажира известны следующие признаки

PassengerId — идентификатор пассажира<br>
Survival — поле в котором указано спасся человек (1) или нет (0)<br>
Pclass — содержит социально-экономический статус (высокий, средний, низкий)<br>
Name — имя пассажира<br>
Sex — пол пассажира<br>
Age — возраст<br>
SibSp — содержит информацию о количестве родственников 2-го порядка (муж, жена, братья, сетры)<br>
Parch — содержит информацию о количестве родственников на борту 1-го порядка (мать, отец, дети)<br>
Ticket — номер билета<br>
Fare — цена билета<br>
Cabin — каюта<br>
Embarked — порт посадки (C — Cherbourg, Q — Queenstown, S — Southampton)<br>

Для начала с помощью Pandas посмотрим на данные

In [ ]:
import pandas as pd
import os

titanic_data = 'titanic_data'

train = pd.read_csv(os.path.join(titanic_data,'train.csv'))
test = pd.read_csv(os.path.join(titanic_data,'test.csv'))

Быстрый старт. Попробуем взять быка за рога

Помним, что модели машинного обучения принимают на вход только числа. Удалим всё, что не число.

In [ ]:
y = train['Survived']

In [ ]:
train.drop(columns=['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked', 'Age'], inplace=True)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(train, y)
predict = model.predict_proba(train)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
from random import random
from matplotlib import pyplot as plt
fpr, tpr, thresholds = roc_curve(y, predict[:,1])

plt.figure(figsize=(17,10))
plt.plot(fpr, tpr)
plt.plot([0,1], [0,1])
print(roc_auc_score(y, predict[:,1]))

Начнём с самого простого - baseline-модели. Например, предположим, что спаслись богатеи из 1 и 2 класса


In [ ]:
predict = []
for i in range(len(train)):
    #predict.append(int((train.loc[i, 'Sex'] == 'female')))
    predict = train['Pclass'].apply(lambda x: int(x < 3))
#predict

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
from matplotlib import pyplot as plt
fpr, tpr, thresholds = roc_curve(y, predict)

plt.figure(figsize=(17,10))
plt.plot(fpr, tpr)
plt.plot([0,1], [0,1])
print(roc_auc_score(y, predict))

Посмотрим теперь, все ли хорошо с данными, есть ли в них пропуски. Очень полезно для этого использовать info() и describe()

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train.groupby('Sex').median()

Видим, что номера кают заполнены очень плохо, возраст можно попытаться как-то восстановить...

In [ ]:
train.Age[train.Age.isnull()]

Остался еще порт посадки - там всего 2 значения не заполнены. Что делать?

In [ ]:
MaxPortEmbarked = train.groupby('Embarked').count()['PassengerId']
MaxPortEmbarked

In [ ]:
MaxPassEmbarked = train.groupby('Embarked').count()['PassengerId']
train.Embarked[train.Embarked.isnull()] = MaxPassEmbarked[MaxPassEmbarked == MaxPassEmbarked.max()].index[0]

In [ ]:
train.Embarked[train.Embarked.isnull()]